In [ ]:
import numpy as np

from src.distrs import Distribution2D, calc_distribution_function_1d, fft_convolve, logger, thurston_volume

In [ ]:
import math
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
alpha = [np.pi / 180 * 60] * 11
dphi = 2e-3
darea = 2e-3

logger.info('setting distribution sizes')

distributions = dict()
for i in range(1, len(alpha) + 1):
    area_max = np.tan(alpha[0] * i / 4) / 2 + 2 * darea
    if area_max <= 0 or area_max > 4000 * darea:
        area_max = 4000 * darea
    distributions[i] = Distribution2D(alpha[:i],
                                      (0, alpha[0] * i), dphi,
                                      (0, area_max), darea,
                                      dtype=np.float64)
for i in distributions:
    print(f'{i}, {distributions[i].distr.shape}')

In [ ]:
distr = distributions[1]
logger.info(f'calculating distribution for {distr.alpha}')
triangle_areas = 1 / (1 / np.tan(distr.phi1 / 2) + 1 / np.tan(distr.phi2 / 2))

indexes = (triangle_areas // distr.darea).astype(int)
indexes = np.minimum(indexes, len(distr.area) - 1)
indexes = np.maximum(indexes, 0)

distr.distr = np.zeros_like(distr.Area)
for i in range(len(distr.distr)):
    index_max = indexes[max(i - 1, 0):min(i + 2, len(distr.distr) - 1)].max()
    index_min = indexes[max(i - 1, 0):min(i + 2, len(distr.distr) - 1)].min()
    if True or index_min < 50:
        distr.distr[i, indexes[i]] += 1 / distr.darea
    else:
        distr.distr[i, index_min:index_max + 1] = 1 / (index_max - index_min + 1) / distr.darea

In [ ]:
distr = distributions[2]
logger.info(f'calculating distribution for {distr.alpha}')
for i, phi1 in enumerate(distr.phi1):
    dist_function = calc_distribution_function_1d(phi1, distr.alpha)
    for j, area in enumerate(distr.area):
        distr.distr[i, j] = dist_function(area)
distr.distr = np.maximum(distr.distr, 0)

In [ ]:
for i in range(3, 11):
    distr = distributions[i]
    logger.info(f'calculating rhs for {distr.alpha}')
    distr.rhs = np.zeros_like(distr.Area)

    for j in range(1, i):
        distr1 = distributions[j]
        distr2 = distributions[i-j]

        logger.info(f'calculating rhs_summand for {distr.alpha}, combination {j}')

        # arg1 = distr1.distr_ * distr1.Q * (1 - distr1.Area * distr1.Q) ** (len(distr1.alpha) - 2)
        # arg2_ = distr2.distr_ * (1 - distr2.Area * distr2.Q) ** (len(distr2.alpha) - 2)

        arg1 = distr1.distr * distr1.Area ** len(distr1.alpha) * distr1.Q
        arg2_ = distr2.distr * distr2.Area ** len(distr2.alpha)

        arg1 = np.nan_to_num(arg1, nan=0, posinf=0, neginf=0)
        arg2_ = np.nan_to_num(arg2_, nan=0, posinf=0, neginf=0)

        arg1[:, :10] = 0
        arg2_[:, :10] = 0

        rhs = np.zeros_like(distr.Area)
        qq = fft_convolve(arg2_, arg1)

        a0 = min(arg1.shape[0] + arg2_.shape[0], rhs.shape[0])
        a1 = min(arg1.shape[1] + arg2_.shape[1], rhs.shape[1])

        rhs[:a0, :a1] = qq[:a0, :a1]
        rhs *= distr2.dphi1 * distr2.darea

        rhs *= math.comb(i, j)

        distr.rhs += rhs

    s = distr.rhs * (1 - distr.Area * distr.Q) ** (1 - len(distr.alpha)) / 2

    s[:, :100] = 0  # todo

    cs = np.cumsum(s, axis=-1) * distr.darea
    cs[(distr.Q > 0) * (distr.Area >= 1/distr.Q)] = 0
    distr.distr_ = cs * (1 - distr.Area * distr.Q) ** (len(distr.alpha) - 2)
    distr.distr = distr.distr_ * distr.Area ** -len(distr.alpha)
    distr.distr = np.nan_to_num(distr.distr, nan=0, posinf=0, neginf=0)

In [ ]:
distr = distributions[8]
_, ax = plt.subplots(1, 2, figsize = (10, 7))
im = ax[0].imshow(np.log(np.maximum(distr.distr, 1e-10)), cmap='hot')
plt.colorbar(im, orientation='horizontal', ax=ax[0])
im = ax[1].imshow(np.log(np.maximum(distr.rhs[:, :], 1e-15)), cmap='hot')
plt.colorbar(im, orientation='horizontal', ax=ax[1])
plt.show()

In [ ]:
distr = distributions[10]

fig = go.Figure()
fig.add_trace(go.Scatter(x=distr.area[:], y=(distr.distr)[1700, :]))
fig.add_trace(go.Scatter(x=distr.area[:], y=(distr.distr)[2100, :]))
fig.add_trace(go.Scatter(x=distr.area[:], y=(distr.distr)[2400, :]))
fig.add_trace(go.Scatter(x=distr.area[:], y=(distr.distr)[len(distr.phi1) // 2, :]))
fig['layout'].update(margin=dict(l=0, r=0, b=0, t=10))
fig.show()

In [ ]:
distr = distributions[7]

fig = go.Figure()
fig.add_trace(go.Scatter(x=distr.phi1[::], y=thurston_volume([2*np.pi - distr.phi1.astype(np.float64), 2*np.pi - distr.phi2.astype(np.float64), *distr.alpha])))
fig.add_trace(go.Scatter(x=distr.phi1[::10], y=distr.distr[::10, 100:].sum(axis=1) * distr.darea))
fig['layout'].update(margin=dict(l=0, r=0, b=0, t=10))
fig.show()

In [ ]:
distr = distributions[4]

fig = go.Figure()
fig.add_trace(go.Scatter(x=distr.area[:], y=(distr.distr)[1500, :]))
fig.add_trace(go.Scatter(x=distr.area[:], y=(distr.distr)[len(distr.phi1) // 2, :]))
fig['layout'].update(margin=dict(l=0, r=0, b=0, t=10))
fig.show()